***
## notebook config

In [1]:
ON_COLAB = False

In [2]:
if ON_COLAB:
    from google.colab import drive
    from google.colab import files
    drive.mount('/content/drive', force_remount=True)
    
    !pip install --upgrade kaggle > /dev/null 2>&1
    !mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
    
    !free -h
    
    !pip install --upgrade category_encoders > /dev/null 2>&1
    !pip install --upgrade jsonlines > /dev/null 2>&1
    !pip install --upgrade pandarallel > /dev/null 2>&1

In [3]:
if ON_COLAB:
    !kaggle datasets download -d mavillan/meli-2021 --force --unzip
    !ls -halt
    input_path = "."
    print("input_path:", input_path)
    output_path = "/content/drive/MyDrive/meli2021/data"
    print("output_path:", output_path)
else:
    input_path = "../data"
    print("input_path:", input_path)
    output_path = "../data"
    print("output_path:", output_path)

input_path: ../data
output_path: ../data


***

In [4]:
import gc
import numpy as np
import pandas as pd
import jsonlines
from tqdm import tqdm
from joblib import Parallel, delayed
import category_encoders as ce
from pandarallel import pandarallel

pd.set_option('display.max_columns', None)
pandarallel.initialize()

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [5]:
def reduce_mem_usage(df, verbose=False):
    """
    Utility function to reduce the memory usage of pandas dataframes
    
    Parameters
    ----------
    df: pandas.Dataframe
    verbose: Boolean
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

***
## preparing train data

In [6]:
train = pd.read_parquet(f"{input_path}/train_data.parquet")
train["date"] = pd.to_datetime(train["date"])
train["weekday"] = (train.date.dt.weekday+1).astype(str)
train

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,weekday
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,1
1,464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,2
2,464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,3
3,464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.000000,4
4,464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.000000,5
...,...,...,...,...,...,...,...,...,...,...
37660274,129187,2021-03-31,0,22057.00,ARG,classic,drop_off,free_shipping,267.710767,3
37660275,6707,2021-03-31,0,26999.00,ARG,classic,cross_docking,free_shipping,266.083333,3
37660276,170355,2021-03-31,0,3400.00,ARG,classic,drop_off,paid_shipping,0.252633,3
37660277,246568,2021-03-31,0,6289.00,ARG,classic,fulfillment,free_shipping,135.416667,3


In [7]:
metadata = pd.read_csv(f"{input_path}/metadata.csv")
metadata

,item_domain_id,item_id,site_id,sku,product_id,product_family_id
0,MLB-SNEAKERS,492155,MLB,0,NaN,MLB15832732
1,MLB-SURFBOARD_RACKS,300279,MLB,1,NaN,NaN
2,MLM-NECKLACES,69847,MLM,2,NaN,NaN
3,MLM-RINGS,298603,MLM,3,NaN,NaN
4,MLB-WEBCAMS,345949,MLB,4,NaN,NaN
...,...,...,...,...,...,...
660911,MLB-CELLPHONE_PARTS,320792,MLB,660911,NaN,NaN
660912,MLM-AUTOMOTIVE_EMBLEMS,90441,MLM,660912,NaN,NaN
660913,MLB-SOFA_AND_FUTON_COVERS,202580,MLB,660913,NaN,NaN
660914,MLB-SNEAKERS,490874,MLB,660914,NaN,NaN


In [8]:
train["tmp"] = train.eval("sold_quantity*minutes_active")
q_mean = (train.groupby("sku")["tmp"].sum() / train.groupby("sku")["minutes_active"].sum()).reset_index(name="q_mean")
train = train.merge(q_mean, how="inner", on="sku")
train.drop("tmp", axis=1, inplace=True)

train["tmp"]  = train.eval("minutes_active * (sold_quantity - q_mean)**2")
q_std = (train.groupby("sku")["tmp"].sum() / train.groupby("sku")["minutes_active"].sum()).reset_index(name="q_std")
train = train.merge(q_std, how="inner", on="sku")
train.drop("tmp", axis=1, inplace=True)

In [9]:
# imputation of zero q_mean & q_std
train = pd.merge(train, metadata.loc[:,["sku","item_domain_id"]], how="inner", on="sku")

df_mean_imp = (
    train
    .loc[:,["sku","item_domain_id","q_mean","q_std"]]
    .drop_duplicates()
    .query("q_mean > 0")
    .groupby("item_domain_id")["q_mean"]
    .mean()
    .reset_index(name="q_mean_imp")
)
df_std_imp = (
    train
    .loc[:,["sku","item_domain_id","q_mean","q_std"]]
    .drop_duplicates()
    .query("q_std > 0")
    .groupby("item_domain_id")["q_std"]
    .median()
    .reset_index(name="q_std_imp")
)

train = (
    train
    .merge(df_mean_imp, how="left", on="item_domain_id")
    .merge(df_std_imp, how="left", on="item_domain_id")
)

idx = train[train.q_mean_imp.isna()].index
train.loc[idx, "q_mean_imp"] = df_mean_imp.q_mean_imp.median()
idx = train[train.q_std_imp.isna()].index
train.loc[idx, "q_std_imp"] = df_std_imp.q_std_imp.median()

idx = train[train.q_mean == 0].index
train.loc[idx, "q_mean"] = train.loc[idx, "q_mean_imp"]
idx = train[train.q_std == 0].index
train.loc[idx, "q_std"] = train.loc[idx, "q_std_imp"]

train.drop(["q_mean_imp","q_std_imp","item_domain_id"], axis=1, inplace=True)

In [10]:
train = reduce_mem_usage(train, verbose=True)
gc.collect()

Mem. usage decreased to 3833.42 Mb (19.6% reduction)


51

In [11]:
train.loc[:,["currency","listing_type","shipping_logistic_type","shipping_payment"]].nunique()

currency                  4
listing_type              2
shipping_logistic_type    3
shipping_payment          2
dtype: int64

In [12]:
encoder = ce.OneHotEncoder(cols=["weekday","listing_type","shipping_logistic_type","shipping_payment"], use_cat_names=True)
train = encoder.fit_transform(train)
train

/Users/mavillan/miniconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,sku,date,sold_quantity,current_price,currency,listing_type_classic,listing_type_premium,shipping_logistic_type_fulfillment,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_payment_free_shipping,shipping_payment_paid_shipping,minutes_active,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,q_mean,q_std
0,464801,2021-02-01,0,156.779999,REA,1,0,1,0,0,1,0,1440.000000,1,0,0,0,0,0,0,0.254237,0.562482
1,464801,2021-02-02,0,156.779999,REA,1,0,1,0,0,1,0,1440.000000,0,1,0,0,0,0,0,0.254237,0.562482
2,464801,2021-02-03,0,156.779999,REA,1,0,1,0,0,1,0,1440.000000,0,0,1,0,0,0,0,0.254237,0.562482
3,464801,2021-02-04,0,156.779999,REA,1,0,1,0,0,1,0,1440.000000,0,0,0,1,0,0,0,0.254237,0.562482
4,464801,2021-02-05,1,156.779999,REA,1,0,1,0,0,1,0,1440.000000,0,0,0,0,1,0,0,0.254237,0.562482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37660274,129187,2021-03-31,0,22057.000000,ARG,1,0,0,0,1,1,0,267.710754,0,0,1,0,0,0,0,0.763131,0.698453
37660275,6707,2021-03-31,0,26999.000000,ARG,1,0,0,1,0,1,0,266.083344,0,0,1,0,0,0,0,2.063710,1.428639
37660276,170355,2021-03-31,0,3400.000000,ARG,1,0,0,0,1,0,1,0.252633,0,0,1,0,0,0,0,0.813597,0.442502
37660277,246568,2021-03-31,0,6289.000000,ARG,1,0,1,0,0,1,0,135.416672,0,0,1,0,0,0,0,2.432324,1.715082


In [13]:
test = pd.read_csv(f"{input_path}/test_data.csv")
train = pd.merge(train, test, how="inner", on="sku")
train = reduce_mem_usage(train, verbose=True)
gc.collect()
train

Mem. usage decreased to 1849.78 Mb (62.7% reduction)


,sku,date,sold_quantity,current_price,currency,listing_type_classic,listing_type_premium,shipping_logistic_type_fulfillment,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_payment_free_shipping,shipping_payment_paid_shipping,minutes_active,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,q_mean,q_std,target_stock
0,464801,2021-02-01,0,156.779999,REA,1,0,1,0,0,1,0,1440.000000,1,0,0,0,0,0,0,0.254237,0.562482,3
1,464801,2021-02-02,0,156.779999,REA,1,0,1,0,0,1,0,1440.000000,0,1,0,0,0,0,0,0.254237,0.562482,3
2,464801,2021-02-03,0,156.779999,REA,1,0,1,0,0,1,0,1440.000000,0,0,1,0,0,0,0,0.254237,0.562482,3
3,464801,2021-02-04,0,156.779999,REA,1,0,1,0,0,1,0,1440.000000,0,0,0,1,0,0,0,0.254237,0.562482,3
4,464801,2021-02-05,1,156.779999,REA,1,0,1,0,0,1,0,1440.000000,0,0,0,0,1,0,0,0.254237,0.562482,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31284422,129187,2021-03-31,0,22057.000000,ARG,1,0,0,0,1,1,0,267.710754,0,0,1,0,0,0,0,0.763131,0.698453,8
31284423,6707,2021-03-31,0,26999.000000,ARG,1,0,0,1,0,1,0,266.083344,0,0,1,0,0,0,0,2.063710,1.428639,30
31284424,170355,2021-03-31,0,3400.000000,ARG,1,0,0,0,1,0,1,0.252633,0,0,1,0,0,0,0,0.813597,0.442502,3
31284425,246568,2021-03-31,0,6289.000000,ARG,1,0,1,0,0,1,0,135.416672,0,0,1,0,0,0,0,2.432324,1.715082,2


In [14]:
cols_sum = [
    "listing_type_classic",
    "listing_type_premium",
    "shipping_logistic_type_fulfillment",
    "shipping_logistic_type_cross_docking",
    "shipping_logistic_type_drop_off",
    "shipping_payment_free_shipping",
    "shipping_payment_paid_shipping",
    "weekday_1",
    "weekday_2",
    "weekday_3",
    "weekday_4",
    "weekday_5",
    "weekday_6",
    "weekday_7",    
]
cols_mean = [
    "minutes_active",
]

def compute_features(df):
    target_stock = df.target_stock.values[0]
    start_index = df.index.min()
    end_index = df.index.max()

    if len(df) < 30:
        return None

    inventory_days = list()
    sum_values = list()
    mean_values = list()

    for i in df.index:
        if i+29 > end_index: break

        df_slice = df.loc[i:i+29]
        cumsum = df_slice.sold_quantity.values.cumsum()
        idxs = np.argwhere(cumsum >= target_stock)
        if len(idxs)==0:
            idays = np.inf
        else:
            idays = np.min(idxs) + 1

        inventory_days.append(idays)
        sum_values.append(df_slice[cols_sum].sum(axis=0).values)
        mean_values.append(df_slice[cols_mean].mean(axis=0).values)

    df = df.head(len(inventory_days)).copy()
    df["inventory_days"] = inventory_days
    df[cols_sum] = np.asarray(sum_values)
    df[cols_mean] = np.asarray(mean_values)
    
    return df

with Parallel(n_jobs=6) as parallel:
    delayed_func = delayed(compute_features)
    all_dfs = parallel(delayed_func(df) for _,df in tqdm(train.groupby("sku")))

100%|██████████| 551472/551472 [1:57:17<00:00, 78.36it/s]  


In [15]:
train = pd.concat(filter(lambda x: x is not None, all_dfs), ignore_index=True)
train

,sku,date,sold_quantity,current_price,currency,listing_type_classic,listing_type_premium,shipping_logistic_type_fulfillment,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_payment_free_shipping,shipping_payment_paid_shipping,minutes_active,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,q_mean,q_std,target_stock,inventory_days
0,0,2021-02-01,0,172.789993,REA,30,0,0,30,0,30,0,1325.088257,5,5,4,4,4,4,4,0.215558,0.482631,1,inf
1,0,2021-02-02,0,172.789993,REA,30,0,0,30,0,30,0,1373.088257,4,5,5,4,4,4,4,0.215558,0.482631,1,inf
2,0,2021-02-03,0,179.990005,REA,30,0,0,30,0,30,0,1421.088257,4,4,5,5,4,4,4,0.215558,0.482631,1,inf
3,0,2021-02-04,0,179.990005,REA,30,0,0,30,0,30,0,1440.000000,4,4,4,5,5,4,4,0.215558,0.482631,1,inf
4,0,2021-02-05,0,179.990005,REA,30,0,0,30,0,30,0,1440.000000,4,4,4,4,5,5,4,0.215558,0.482631,1,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15524803,660914,2021-02-26,1,59.990002,REA,0,30,30,0,0,0,30,1440.000000,4,4,4,4,5,5,4,0.423169,0.759259,5,8.0
15524804,660914,2021-02-27,0,59.990002,REA,0,30,30,0,0,0,30,1440.000000,4,4,4,4,4,5,5,0.423169,0.759259,5,9.0
15524805,660914,2021-02-28,1,59.990002,REA,0,30,30,0,0,0,30,1440.000000,5,4,4,4,4,4,5,0.423169,0.759259,5,8.0
15524806,660914,2021-03-01,0,59.990002,REA,0,30,30,0,0,0,30,1440.000000,5,5,4,4,4,4,4,0.423169,0.759259,5,9.0


In [16]:
train = pd.merge(train, metadata, how="inner", on="sku")
train

,sku,date,sold_quantity,current_price,currency,listing_type_classic,listing_type_premium,shipping_logistic_type_fulfillment,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_payment_free_shipping,shipping_payment_paid_shipping,minutes_active,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,q_mean,q_std,target_stock,inventory_days,item_domain_id,item_id,site_id,product_id,product_family_id
0,0,2021-02-01,0,172.789993,REA,30,0,0,30,0,30,0,1325.088257,5,5,4,4,4,4,4,0.215558,0.482631,1,inf,MLB-SNEAKERS,492155,MLB,NaN,MLB15832732
1,0,2021-02-02,0,172.789993,REA,30,0,0,30,0,30,0,1373.088257,4,5,5,4,4,4,4,0.215558,0.482631,1,inf,MLB-SNEAKERS,492155,MLB,NaN,MLB15832732
2,0,2021-02-03,0,179.990005,REA,30,0,0,30,0,30,0,1421.088257,4,4,5,5,4,4,4,0.215558,0.482631,1,inf,MLB-SNEAKERS,492155,MLB,NaN,MLB15832732
3,0,2021-02-04,0,179.990005,REA,30,0,0,30,0,30,0,1440.000000,4,4,4,5,5,4,4,0.215558,0.482631,1,inf,MLB-SNEAKERS,492155,MLB,NaN,MLB15832732
4,0,2021-02-05,0,179.990005,REA,30,0,0,30,0,30,0,1440.000000,4,4,4,4,5,5,4,0.215558,0.482631,1,inf,MLB-SNEAKERS,492155,MLB,NaN,MLB15832732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15524803,660914,2021-02-26,1,59.990002,REA,0,30,30,0,0,0,30,1440.000000,4,4,4,4,5,5,4,0.423169,0.759259,5,8.0,MLB-SNEAKERS,490874,MLB,NaN,NaN
15524804,660914,2021-02-27,0,59.990002,REA,0,30,30,0,0,0,30,1440.000000,4,4,4,4,4,5,5,0.423169,0.759259,5,9.0,MLB-SNEAKERS,490874,MLB,NaN,NaN
15524805,660914,2021-02-28,1,59.990002,REA,0,30,30,0,0,0,30,1440.000000,5,4,4,4,4,4,5,0.423169,0.759259,5,8.0,MLB-SNEAKERS,490874,MLB,NaN,NaN
15524806,660914,2021-03-01,0,59.990002,REA,0,30,30,0,0,0,30,1440.000000,5,5,4,4,4,4,4,0.423169,0.759259,5,9.0,MLB-SNEAKERS,490874,MLB,NaN,NaN


In [17]:
train = reduce_mem_usage(train, verbose=True)
train.to_parquet("../data/train-m2.parquet", index=False)

Mem. usage decreased to 1569.39 Mb (49.0% reduction)


***
## preparing test data

***